In [1]:
import networkx as nx
import numpy as np
import random
from math import floor

def colorful_triangle_estimate(G, p, trials=10):
    """
    Estimate the number of triangles using the colorful triangle counting algorithm.
    
    Parameters:
    - G: undirected NetworkX graph
    - p: sampling probability, used to set C = floor(1/p)
    - trials: number of independent repetitions
    
    Returns:
    - mean and std deviation of triangle estimates
    """
    C = max(1, floor(1/p))
    estimates = []
    
    nodes = list(G.nodes())
    
    for _ in range(trials):
        # Assign each node a random color in {0, ..., C-1}
        color = {u: random.randint(0, C - 1) for u in nodes}
        
        # Induce the monochromatic subgraph
        H = nx.Graph()
        H.add_nodes_from(nodes)
        for u, v in G.edges():
            if color[u] == color[v]:
                H.add_edge(u, v)
        
        # Count triangles in H and scale
        triangle_count = sum(nx.triangles(H).values()) // 3
        estimate = triangle_count * (C**2)
        estimates.append(estimate)
    
    return np.mean(estimates), np.std(estimates)


In [4]:
# === Apply on G(n, p) ===
n = 100
p_edge = 0.5
G_random = nx.erdos_renyi_graph(n, p_edge)

# True triangle count for comparison
true_triangles_random = sum(nx.triangles(G_random).values()) // 3

# Run colorful triangle estimation
p_sample = 0.4  # sampling probability (sets C = floor(1/p) = 5)
est_mean_rand, est_std_rand = colorful_triangle_estimate(G_random, p_sample, trials=20)

print("\nG(n, p) Random Graph")
print(f"True triangle count: {true_triangles_random}")
print(f"Colorful triangle estimate (mean ± std): {est_mean_rand:.1f} ± {est_std_rand:.1f}")


# === Example usage on Davis Women projection ===
G_bipartite = nx.davis_southern_women_graph()
women_nodes = [n for n, d in G_bipartite.nodes(data=True) if d["bipartite"] == 0]
G_women = nx.bipartite.projected_graph(G_bipartite, women_nodes)

true_triangle_count = sum(nx.triangles(G_women).values()) // 3
p = 0.3
est_mean, est_std = colorful_triangle_estimate(G_women, p, trials=20)

print(f"True triangle count: {true_triangle_count}")
print(f"Colorful triangle estimate (mean ± std): {est_mean:.1f} ± {est_std:.1f}")



G(n, p) Random Graph
True triangle count: 21381
Colorful triangle estimate (mean ± std): 21470.8 ± 1327.5
True triangle count: 631
Colorful triangle estimate (mean ± std): 626.0 ± 214.8
